In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.io import loadmat
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve

!pip install -U mne
# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

!pip install pyriemann
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
#from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression




In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
#file_path = '/content/drive/MyDrive/MyData/your_file.csv'
# # read data and use a correct comand based on your format
# import pandas as pd
# data = pd.read_csv(file_path)

Mounted at /content/drive


In [ ]:
data_folder_path = "/content/drive/MyDrive/Colab Notebooks/EEG_Data_Large_Scale"
Tr_Indx_mat_file = data_folder_path + "/" + "Index_10Runs_Trains.mat"
Indices_Tr_Load=loadmat(Tr_Indx_mat_file)
Indices_Train=Indices_Tr_Load["IDX_train"]

Te_Indx_mat_file = data_folder_path + "/" + "Index_10Runs_Tests.mat"
Indices_Te_Load=loadmat(Te_Indx_mat_file)
Indices_Test=Indices_Te_Load["IDX_test"]

In [ ]:
data_list = os.listdir(data_folder_path)
data_list = data_list[3:]
data_list.pop(28)
data_list.pop(33)
data_list.remove("EEGModels.py")
data_list
 #       file_path = os.path.join(data_folder_path, file_name)



['Subject1_Signal_Downsampled.mat',
 'Subject2_Signal_Downsampled.mat',
 'Subject3_Signal_Downsampled.mat',
 'Subject4_Signal_Downsampled.mat',
 'Subject5_Signal_Downsampled.mat',
 'Subject6_Signal_Downsampled.mat',
 'Subject7_Signal_Downsampled.mat',
 'Subject8_Signal_Downsampled.mat',
 'Subject9_Signal_Downsampled.mat',
 'Subject10_Signal_Downsampled.mat',
 'Subject11_Signal_Downsampled.mat',
 'Subject12_Signal_Downsampled.mat',
 'Subject13_Signal_Downsampled.mat',
 'Subject14_Signal_Downsampled.mat',
 'Subject15_Signal_Downsampled.mat',
 'Subject16_Signal_Downsampled.mat',
 'Subject17_Signal_Downsampled.mat',
 'Subject18_Signal_Downsampled.mat',
 'Subject19_Signal_Downsampled.mat',
 'Subject20_Signal_Downsampled.mat',
 'Subject21_Signal_Downsampled.mat',
 'Subject22_Signal_Downsampled.mat',
 'Subject23_Signal_Downsampled.mat',
 'Subject24_Signal_Downsampled.mat',
 'Subject25_Signal_Downsampled.mat',
 'Subject26_Signal_Downsampled.mat',
 'Subject27_Signal_Downsampled.mat',
 'Subject2

In [ ]:
# Specify the path to your MATLAB file
#mat_file_path = 'F:\EEG_procesing_workshop\EEG_DL\Codes\CODE2_EEG_motor_im_DFFNN\saved_mat_data\motor_im_feature.mat'
#mat_file_path = data_folder_path + "/" + "Index_10Runs.mat"

for i_subj in data_list[1:]:
  # i_subj=data_list[i_subj]
  # i_subj=data_list[3]
  # Load Subject data
  file_path = os.path.join(data_folder_path, i_subj)
  mat_data = loadmat(file_path)
  Size1_1=mat_data["SIG_Downsampled"][0].size
  Size2_1=mat_data["SIG_Downsampled"][0][1].shape
  Size1_2=mat_data["SIG_Downsampled"][1].size
  Size2_2=mat_data["SIG_Downsampled"][1][1].shape
  print(Size1_1)
  print(Size2_1)
  print(Size1_2)
  print(Size2_2)
  Sig_1 = np.ndarray((Size1_1,Size2_1[1],Size2_1[0]))
  Sig_2 = np.ndarray((Size1_2,Size2_2[1],Size2_2[0]))
  for i in range(Size1_1):
    # print(mat_data["SIG_Downsampled"][0][i].shape)
    if mat_data["SIG_Downsampled"][0][i].shape==Size2_1:
       Sig_1[i] = np.transpose(mat_data["SIG_Downsampled"][0][i])
  for i in range(Size1_2):
    if mat_data["SIG_Downsampled"][1][i].shape==Size2_2:
       Sig_2[i] = np.transpose(mat_data["SIG_Downsampled"][1][i])
  i_run=1
  i_fold=1
  Sig_Train_1 = np.squeeze(Sig_1[Indices_Train[i_run][i_fold][0]-1])
  Sig_Test_1 = np.squeeze(Sig_1[Indices_Test[i_run][i_fold][0]-1])
  Sig_Train_2 = np.squeeze(Sig_2[Indices_Train[i_run][i_fold][1]-1])
  Sig_Test_2 = np.squeeze(Sig_1[Indices_Test[i_run][i_fold][1]-1])
  X_train=np.concatenate((Sig_Train_1[0:round(0.8*Size1_1)],Sig_Train_2[0:round(0.8*Size1_2)]),axis=0)
  Y_train=np.concatenate((np.ones(round(0.8*Size1_1)),2*np.ones(round(0.8*Size1_2))),axis=0)
  X_validate=np.concatenate((Sig_Train_1[round(0.8*Size1_1):round(0.9*Size1_1)],Sig_Train_2[round(0.8*Size1_2):round(0.9*Size1_2)]),axis=0)
  Y_validate=np.concatenate((np.ones(round(0.1*Size1_1)),2*np.ones(round(0.1*Size1_2))),axis=0)
  X_test=np.concatenate((Sig_Test_1,Sig_Test_2),axis=0)
  Y_test=np.concatenate((np.ones(round(0.1*Size1_1)),2*np.ones(round(0.1*Size1_2))),axis=0)
  kernels, chans, samples = 1, 64, 257
  ##################################################################################################################
  # EEGNet Model
  # convert labels to one-hot encodings.
  Y_train      = np_utils.to_categorical(Y_train-1)
  Y_validate   = np_utils.to_categorical(Y_validate-1)
  Y_test       = np_utils.to_categorical(Y_test-1)
  # convert data to NHWC (trials, channels, samples, kernels) format. Data
  # contains 60 channels and 151 time-points. Set the number of kernels to 1.
  X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
  X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
  X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
  print('X_train shape:', X_train.shape)
  print(X_train.shape[0], 'train samples')
  print(X_test.shape[0], 'test samples')
  # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other
  # model configurations may do better, but this is a good starting point)
  model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples,
                 dropoutRate = 0.5, kernLength = 64, F1 = 8, D = 2, F2 = 16,
                 dropoutType = 'Dropout')
  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics = ['accuracy'])
  # count number of parameters in the model
  numParams    = model.count_params()
  # set a valid path for your system to record model checkpoints
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                 save_best_only=True)
  ###############################################################################
  # if the classification task was imbalanced (significantly more trials in one
  # class versus the others) you can assign a weight to each class during
  # optimization to balance it out. This data is approximately balanced so we
  # don't need to do this, but is shown here for illustration/completeness.
  ###############################################################################
  # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
  # the weights all to be 1
  class_weights = {0:1, 1:1, 2:1, 3:1}
  ################################################################################
  # fit the model. Due to very small sample sizes this can get
  # pretty noisy run-to-run, but most runs should be comparable to xDAWN +
  # Riemannian geometry classification (below)
  ################################################################################
  fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 3,
                          verbose = 2, validation_data=(X_validate, Y_validate),
                         callbacks=[checkpointer], class_weight = class_weights)
  #fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300,
  #                        verbose = 2,
  #                        callbacks=[checkpointer], class_weight = class_weights)
  # load optimal weights
  # model.load_weights('/tmp/checkpoint.h5')
  ###############################################################################
  # can alternatively used the weights provided in the repo. If so it should get
  # you 93% accuracy. Change the WEIGHTS_PATH variable to wherever it is on your
  # system.
  ###############################################################################
  # WEIGHTS_PATH = /path/to/EEGNet-8-2-weights.h5
  # model.load_weights(WEIGHTS_PATH)
  ###############################################################################
  # make prediction on test set.
  ###############################################################################
  probs       = model.predict(X_test)
  preds       = probs.argmax(axis = -1)
  acc         = np.mean(preds == Y_test.argmax(axis=-1))
  print("Classification accuracy: %f " % (acc))
  # result_folder_path = "/content/drive/MyDrive/Colab Notebooks/Results"
  # result_accuracy_file = os.path.join(result_folder_path, i_subj[:-3]+"txt")
  # f = open(result_accuracy_file, "a")
  # f.write(str(acc))
  # f.close()
  result_folder_path = "/content/drive/MyDrive/Colab Notebooks/Results/all_result1.txt"
  f = open(result_folder_path, "a")
  f.write("\n"+str(acc))
  f.close()

#%whos


In [ ]:
# from sklearn.model_selection import KFold, cross_val_score
# k_folds = KFold(n_splits = 5,shuffle= True)

import random
mylist=list(range(121))
random.shuffle(mylist)
print(mylist)

%whos

[51, 38, 73, 65, 9, 39, 107, 12, 81, 1, 4, 63, 117, 56, 115, 34, 13, 96, 35, 2, 32, 25, 20, 52, 29, 66, 40, 86, 72, 116, 104, 82, 102, 36, 112, 109, 113, 21, 24, 42, 47, 33, 90, 43, 108, 8, 3, 79, 87, 100, 98, 11, 15, 67, 18, 88, 89, 111, 75, 120, 7, 114, 64, 5, 84, 69, 97, 55, 48, 105, 26, 53, 103, 31, 44, 74, 10, 92, 45, 76, 110, 99, 80, 60, 77, 22, 118, 57, 119, 93, 85, 19, 23, 17, 71, 62, 61, 58, 94, 70, 30, 41, 49, 14, 101, 37, 46, 78, 27, 68, 91, 54, 59, 95, 28, 83, 0, 6, 106, 50, 16]
Variable                 Type               Data/Info
-----------------------------------------------------
Adam                     type               <class 'keras.src.optimizers.adam.Adam'>
EEGNet                   function           <function EEGNet at 0x7f613815ce50>
EarlyStopping            type               <class 'keras.src.callbacks.EarlyStopping'>
Indices_Te_Load          dict               n=4
Indices_Test             ndarray            10x10x2: 200 elems, type `object`, 1600 bytes
Indic

# EEGNet

## Import Libraries

### Please add EEGmodels.py near your file

### data loading

### EEGNet Prepration

In [ ]:

# convert labels to one-hot encodings.
Y_train      = np_utils.to_categorical(Y_train-1)
Y_validate   = np_utils.to_categorical(Y_validate-1)
Y_test       = np_utils.to_categorical(Y_test-1)

# convert data to NHWC (trials, channels, samples, kernels) format. Data
# contains 60 channels and 151 time-points. Set the number of kernels to 1.
X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples,
               dropoutRate = 0.5, kernLength = 64, F1 = 8, D = 2, F2 = 16,
               dropoutType = 'Dropout')

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

###############################################################################
# if the classification task was imbalanced (significantly more trials in one
# class versus the others) you can assign a weight to each class during
# optimization to balance it out. This data is approximately balanced so we
# don't need to do this, but is shown here for illustration/completeness.
###############################################################################

# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1, 2:1, 3:1}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN +
# Riemannian geometry classification (below)
################################################################################
fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300,
                        verbose = 2, validation_data=(X_validate, Y_validate),
                       callbacks=[checkpointer], class_weight = class_weights)

#fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300,
#                        verbose = 2,
#                        callbacks=[checkpointer], class_weight = class_weights)


# load optimal weights
# model.load_weights('/tmp/checkpoint.h5')

###############################################################################
# can alternatively used the weights provided in the repo. If so it should get
# you 93% accuracy. Change the WEIGHTS_PATH variable to wherever it is on your
# system.
###############################################################################

# WEIGHTS_PATH = /path/to/EEGNet-8-2-weights.h5
# model.load_weights(WEIGHTS_PATH)

###############################################################################
# make prediction on test set.
###############################################################################

probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)
acc         = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))




X_train shape: (100, 64, 257, 1)
100 train samples
50 test samples
Epoch 1/300

Epoch 1: val_loss improved from inf to 0.69325, saving model to /tmp/checkpoint.h5
7/7 - 4s - loss: 0.6892 - accuracy: 0.4700 - val_loss: 0.6933 - val_accuracy: 0.5000 - 4s/epoch - 521ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.69325 to 0.69283, saving model to /tmp/checkpoint.h5
7/7 - 3s - loss: 0.7040 - accuracy: 0.4300 - val_loss: 0.6928 - val_accuracy: 0.5200 - 3s/epoch - 442ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.69283 to 0.69255, saving model to /tmp/checkpoint.h5
7/7 - 4s - loss: 0.6925 - accuracy: 0.5500 - val_loss: 0.6926 - val_accuracy: 0.5200 - 4s/epoch - 511ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.69255 to 0.69224, saving model to /tmp/checkpoint.h5
7/7 - 2s - loss: 0.6991 - accuracy: 0.4200 - val_loss: 0.6922 - val_accuracy: 0.6000 - 2s/epoch - 290ms/step
Epoch 5/300

Epoch 5: val_loss improved from 0.69224 to 0.69220, saving model to /tmp/checkpoint.h5
7

In [ ]:
############################# PyRiemann Portion ##############################

# code is taken from PyRiemann's ERP sample script, which is decoding in
# the tangent space with a logistic regression

n_components = 2  # pick some components

# set up sklearn pipeline
from sklearn.pipeline import make_pipeline

clf = make_pipeline(XdawnCovariances(n_components),
                    TangentSpace(metric='riemann'),
                    LogisticRegression())

preds_rg     = np.zeros(len(Y_test))

# reshape back to (trials, channels, samples)
X_train      = X_train.reshape(X_train.shape[0], chans, samples)
X_test       = X_test.reshape(X_test.shape[0], chans, samples)

# train a classifier with xDAWN spatial filtering + Riemannian Geometry (RG)
# labels need to be back in single-column format
clf.fit(X_train, Y_train.argmax(axis = -1))
preds_rg     = clf.predict(X_test)

# Printing the results
acc2         = np.mean(preds_rg == Y_test.argmax(axis = -1))
print("Classification accuracy: %f " % (acc2))

# plot the confusion matrices for both classifiers


Classification accuracy: 0.944444 


In [ ]:
from sklearn import metrics
names        = ['audio left', 'audio right', 'vis left', 'vis right']
plt.figure(1)
#plot_confusion_matrix(preds_rg, Y_test.argmax(axis = -1), names, title = 'xDAWN + RG')

metrics.confusion_matrix(Y_test.argmax(axis = -1), preds)


array([[13, 12],
       [ 9, 16]])

<Figure size 640x480 with 0 Axes>